In [1]:
import cv2

In [2]:
#Here to Distinct the moving object we use absolute difference rather than KNN or MOG2 background
#Subtracter as it gives beeter results than other two.

print("Give Path of the Video")
path = input()

print("Enter x1, y1, x2, y2 Coordinates")
x1 = input("x1 :")
x1 = int(x1)
y1 = input("y1 :")
y1 = int(y1)
x2 = input("x2 :")
x2 = int(x2)
y2 = input("y2 :")
y2 = int(y2)

Give Path of the Video
Night Time Traffic Camera video.mp4
Enter x1, y1, x2, y2 Coordinates
x1 :1
y1 :300
x2 :1000
y2 :300


In [3]:
VidCap = cv2.VideoCapture(path)
ret, frame1 = VidCap.read()   #1st Frame

frame_width = int(VidCap.get(3))
frame_height = int(VidCap.get(4))
   
size = (frame_width, frame_height)
result = cv2.VideoWriter('Ans4.avi',cv2.VideoWriter_fourcc(*'MJPG'), 10, size)

vehicle = 0  #initialize count of vehicle = 0

while VidCap.isOpened():
    ret, frame2 = VidCap.read()  #2nd Frame
    
    if not ret:   #ret return false when video gets over.
        break
        
    fgMask = cv2.absdiff(frame1, frame2)  #Using Absolute Difference.
    fgMask = cv2.cvtColor(fgMask, cv2.COLOR_BGR2GRAY)  #Coverting to gray
    _, thres = cv2.threshold(fgMask, 127, 255, cv2.THRESH_BINARY)  #Applying threshold to abs. diff
    frame1 = frame2  #Swapping frames for video to continues running
    
    cv2.line(frame2, (x1, y1), (x2, y2), (0, 0, 255), 3)       #main Line
    cv2.line(frame2, (x1, y1+10), (x2, y2+10), (0, 255, 0), 2) #offset Line
    cv2.line(frame2, (x1, y1-10), (x2, y2-10), (0, 255, 0), 2) #offset line
    
    conts, _ = cv2.findContours(thres, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    
    for c in conts:
        if cv2.contourArea(c) < 500:   #To Avoid noise.
            continue
        
        x, y, w, h = cv2.boundingRect(c)  #Coordinates of Rectangle.
        cv2.rectangle(fgMask, (x, y), (x+w, y+h), (0, 255, 0), 3)
        xMid = int((x + (x+w))/2)  #Mid point of vehicle around x
        yMid = int((y + (y+w))/2)  # Mid point of vehicle around y
        cv2.circle(fgMask, (xMid, yMid), 5, (0, 0, 255), 5)
        
        if yMid > y1-10 and yMid < y1+10:
            vehicle += 1
            
        cv2.imshow('Foreground Mask', fgMask)
        cv2.putText(fgMask, 'Total Vehicle {}'.format(vehicle), (450, 50), cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 255), 2)
        
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cv2.destroyAllWindows()
VidCap.release()
result.release()